In [166]:
import numpy as np
import csv
import sys

np.set_printoptions(edgeitems=10)

###Code for writing submission file

In [187]:
with open('data/test.txt') as csvfile:
    reader = csv.reader(csvfile)
    reader.next()
    with open ('submission.txt', 'w') as f:
        counter = 1
        f.write('Index,Count\n')
        for row in reader:
            f.write('{},{}\n'.format(counter, agg[(int(row[1])%10000)/100, int(row[0][1:])]))
            counter += 1

###Import data

In [224]:
data = np.loadtxt('data/train.txt', dtype=int, delimiter=',', skiprows=1)

###Fill data with row means

In [105]:
def fill(data, method='rowmean', roundtype='ceil'):
    if method == 'rowmean':
        temp = data        
        for i in range(data.shape[0]):
            row_mean = temp[i, 1:][temp[i, 1:]!=-1].mean()
            if roundtype == 'ceil':
                row_mean = np.ceil(row_mean)
            elif roundtype == 'floor':
                row_mean = np.floor(row_mean)
            np.place(data[i], data[i]==-1, row_mean)
    
    return data

In [107]:
a = fill(data)
s = np.zeros((a.shape[0] / 60., a.shape[1]), dtype=int)

for i in range(s.shape[0]):
    s[i, 0] = a[(i+1)*60 - 1, 0] % 10000
    s[i, 1:] = np.sum(a[i*60:(i+1)*60, 1:], axis=0)

In [159]:
agg = np.zeros((24, a.shape[1]-1), dtype=int) # avg value for each sensor for each hour
for i in range(s.shape[0]):
    agg[(s[i, 0]-59) / 100] += s[i, 1:]
agg /= 30

###Adjacency List

In [191]:
with open('data/sensor-coordinates.txt') as f:
    coords = list()
    reader = csv.reader(f)
    reader.next()
    for row in reader:
        coords.append((float(row[1]), float(row[2])))

In [193]:
def dist(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

In [240]:
def get_nearest(coords, r=10):
    nearest = list()
    for i in range(56):
        temp = list()
        for j in range(56):
            if i != j:
                if dist(coords[i], coords[j]) < r:
                    temp.append(j+1)
        nearest.append(i+1)
        nearest.append(tuple(temp))
    return nearest

In [241]:
n = get_nearest(coords)